In [1]:
'''
This script trains the cell segmentation U-Net

@author: jblugagne
Small augmentations: georgeoshardo
'''
# pip install "scikit_image==0.16.2"
#conda install -c conda-forge tensorflow-gpu=1.15
from SyMBac.external.DeLTA.model import unet_seg
from SyMBac.external.DeLTA.data import trainGenerator_seg
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from random import shuffle
import glob
import os
import random
from shutil import copyfile, rmtree

In [2]:

DeLTA_data =  "/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/delta_hist_mid_large/"

training_set = DeLTA_data
model_file = DeLTA_data + 'models/saved-model-varnum-{number:02d}.hdf5' #'/unet.hdf5'

In [3]:
img_path = training_set+'preprocessed/CROPPED_FILTERED/'
mask_path = training_set+'preprocessed/CROPPED_MASKS/'
weight_path = training_set + 'preprocessed/WEIGHTMAPS/'

In [4]:
temp_store = "/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/temp/"
try:
    os.mkdir(temp_store)
except:
    pass
img_path_tmp = temp_store+'/CROPPED_FILTERED/'
mask_path_tmp = temp_store+'/CROPPED_MASKS/'
weight_path_tmp = temp_store + '/WEIGHTMAPS/'

try:
    os.mkdir(img_path_tmp)
    os.mkdir(mask_path_tmp)
    os.mkdir(weight_path_tmp)
except:
    pass

In [5]:
try:
    rmtree(img_path_tmp)
    rmtree(mask_path_tmp)
    rmtree(weight_path_tmp)
except:
    pass

In [6]:
2000*0.0005

1.0

In [7]:
for frac in [0.0005,0.001,0.005, 0.01, 0.1, 0.2, 0.5, 0.75, 0.9]:
    os.mkdir(img_path_tmp)
    os.mkdir(mask_path_tmp)
    os.mkdir(weight_path_tmp)
    training_image_names = os.listdir(img_path)
    shuffle(training_image_names)
    sample_amount = int(frac*len(training_image_names))
    sample = training_image_names[:sample_amount]

    for perm_path, tmp_path in zip([img_path, mask_path, weight_path],[img_path_tmp,mask_path_tmp,weight_path_tmp]):
        copy_from = [perm_path + img_name for img_name in sample]
        copy_to = [tmp_path + img_name for img_name in sample]
        for frm, to in zip(copy_from, copy_to):
            copyfile(frm,to)


    model_file = DeLTA_data + 'models/saved-model-frac-{}.hdf5'.format(frac) #'/unet.hdf5'
    # Parameters:
    target_size = (192,32)
    input_size = target_size + (1,)
    batch_size = 10
    epochs = 15
    steps_per_epoch = 250

    #Data generator:
    data_gen_args = dict(
                        rotation = 0.5,
                        shiftX=.08,
                        shiftY=.08,
                        zoom=.17,
                        horizontal_flip=True,
                        histogram_voodoo=True,
                        illumination_voodoo=True)

    myGene = trainGenerator_seg(batch_size,
                               img_path_tmp,
                               mask_path_tmp,
                               weight_path_tmp,
                               augment_params = data_gen_args,
                               target_size = target_size)


    # Define model:
    model = unet_seg(input_size = input_size)
    model.summary()
    model_checkpoint = ModelCheckpoint(model_file, monitor='loss',verbose=1, save_best_only=False)

    #filepath = "saved-model-{epoch:02d}-{loss:.2f}.hdf5"
    # Train it:
    model.fit_generator(myGene,steps_per_epoch=steps_per_epoch,epochs=epochs,callbacks=[model_checkpoint])

    rmtree(img_path_tmp)
    rmtree(mask_path_tmp)
    rmtree(weight_path_tmp)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
true_input (InputLayer)         [(None, 192, 32, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 192, 32, 64)  640         true_input[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 192, 32, 64)  36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 96, 16, 64)   0           conv2d_1[0][0]                   
______________________________________________________________________________________________

/home/georgeos/anaconda3/envs/stardist-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/delta/data.py:91: FutureWarning: Input image dtype is bool. Gaussian convolution is not defined with bool data type. Please set anti_aliasing to False or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  i = trans.resize(i, target_size, anti_aliasing=True, order=order)


250/250 [==============================] - 25s 88ms/step - loss: 0.0013 - unstack_acc: 0.8729

Epoch 00001: saving model to /home/georgeos/Storage/Dropbox (Cambridge University)/Phase/delta_hist_mid_large/models/saved-model-frac-0.0005.hdf5
Epoch 2/15
250/250 [==============================] - 22s 87ms/step - loss: 3.8837e-04 - unstack_acc: 0.9658

Epoch 00002: saving model to /home/georgeos/Storage/Dropbox (Cambridge University)/Phase/delta_hist_mid_large/models/saved-model-frac-0.0005.hdf5
Epoch 3/15
250/250 [==============================] - 22s 87ms/step - loss: 3.1985e-04 - unstack_acc: 0.9701

Epoch 00003: saving model to /home/georgeos/Storage/Dropbox (Cambridge University)/Phase/delta_hist_mid_large/models/saved-model-frac-0.0005.hdf5
Epoch 4/15
250/250 [==============================] - 22s 87ms/step - loss: 2.9171e-04 - unstack_acc: 0.9723

Epoch 00004: saving model to /home/georgeos/Storage/Dropbox (Cambridge University)/Phase/delta_hist_mid_large/models/saved-model-frac-0.0